In [1]:
import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

from datasets import load_dataset
from transformer_lens.utils import tokenize_and_concatenate

import tools
import latent_features

2025-02-16 14:33:06.778455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739716386.792996 1833469 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739716386.797649 1833469 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 14:33:06.813613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [2]:
model = HookedTransformer.from_pretrained("gpt2-small", fold_ln=True)

layer_index = 6
location = "mlp_post_act"
transformer_lens_loc = f"blocks.{layer_index}.mlp.hook_post"
prev_layer_loc = f"blocks.{layer_index}.ln2.hook_normalized"

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/vol/bitbucket/mwl21/sae-qaf-venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
ds = load_dataset("NeelNanda/pile-10k", split='train[:10]')
ds_tokens = model.to_tokens(ds['text'])
ds_logits, ds_cache = model.run_with_cache(ds_tokens[0])

In [ ]:
mlp = tools.extract_mlp(model, layer_index)

ds_acts = ds_cache[prev_layer_loc].numpy().reshape(-1, 768)
original_mlp = tools.get_original_mlp_for_sparx(mlp, ds_acts)

shrink_pcs = [0.1, 0.3, 0.5, 0.7, 0.9]
merged_models = []
models_cluster_labels = []
for pc in shrink_pcs:
  model, labels = tools.shrink_model_global(original_mlp, pc)
  model.model.summary()
  merged_models.append(model)
  models_cluster_labels.append(labels)

In [ ]:
from tools import output_infidelity, output_mse, output_r2

# check fidelity and error (MSE)
original_mlp.forward_pass(ds_acts)
original_output = original_mlp.forward_pass_data[-1]

for merged_model in merged_models:
  merged_model.forward_pass(ds_acts)
  merged_output = merged_model.forward_pass_data[-1]

  print(f"infidelity: {output_infidelity(original_output, merged_output)}")
  print(f"MSE error: {output_mse(original_output, merged_output)}")
  print(f"R2: {output_r2(original_output, merged_output)}")

In [ ]:
autoencoder = tools.get_sparse_autoencoder(location, layer_index)

In [ ]:
import numpy as np

sim_matrices = []

for model, labels in zip(merged_models, models_cluster_labels):
  print(f"Average similarity for sparsity {shrink_pcs[len(sim_matrices)]}")

  merged_w_in = model.model.get_weights()[0]
  num_clusters = merged_w_in.shape[1]
  # Get map of cluster index to list of latent feature indices for that cluster
  cluster_latents = latent_features.get_cluster_latents(model, labels, ds_cache)
  # Build a matrix comparing proportion of shared latent concepts between every cluster
  cluster_sim = latent_features.get_cluster_similarity_matrix(num_clusters, cluster_latents)
  sim_matrices.append(cluster_sim)
  average_similarity_score = np.mean(cluster_sim)
  print(average_similarity_score)

In [ ]:
for sim_matrix in sim_matrices:
  latent_features.plot_cluster_similarity_matrix(sim_matrix)